In [4]:
%load_ext autoreload
%autoreload 2

In [5]:

from dcodebeauty.datos import read_file
from dcodebeauty.pre_utils import find_description
from dcodebeauty.ocr import find_product_description,detect_text,cleaning
from gensim.models.ldamodel import LdaModel
from gensim_func import id2word,corpus

import pandas as pd
import numpy as np

In [2]:
ruta_modelo = '../models/model_3_topics'
model = LdaModel.load(ruta_modelo)
corpus = corpus

In [3]:
text = 'moisture bond refreshing hydra-gel oil free'
product = find_product_description(text)
clean_product = cleaning(product)
description = find_description(clean_product)

Found 71.93% of ingredients


In [47]:
description['descriptions'][2]
print(description['ingredients'][2])

Hydrogenated Polyisobutene


In [76]:
values = [model[id2word.doc2bow(description['descriptions'][i].split())] for i in range(len(description['descriptions']))]
df_topic = pd.DataFrame(values)
df_topic['Ingredient'] = description['ingredients']
df_topic = df_topic.rename(columns={0:'quimico',1:"efecto",2:"natural"})
df_topic = df_topic.applymap(lambda x: (0,0) if x is None else x)
df_topic['quimico'] = df_topic['quimico'].apply(lambda x: x[1])
df_topic['efecto'] = df_topic['efecto'].apply(lambda x: x[1])
df_topic['natural'] = df_topic['natural'].apply(lambda x: x[1])
df_topic = df_topic.set_index('Ingredient')

In [ ]:
df_topic

In [84]:
conds = [
    (df_topic['quimico'] > df_topic['efecto']) & (df_topic['quimico'] > df_topic['natural']),
    (df_topic['natural'] > df_topic['efecto']) & (df_topic['natural'] > df_topic['quimico']),
    (df_topic['efecto'] > df_topic['quimico']) & (df_topic['efecto'] > df_topic['natural'])
]

leters = ['quimico','natural','efecto']

df_topic['resultado_parcial'] = np.select(conds,leters)
df_topic

 

,quimico,efecto,natural,resultado_parcial,resultado_final
Ingredient,,,,,
Butylene Glycol,0.030778,0.898876,0.070346,efecto,no se puede determinar
Cyclopentasiloxane,0.379543,0.269364,0.351093,quimico,no se puede determinar
Hydrogenated Polyisobutene,0.283513,0.547903,0.168583,efecto,quimico
Glycerin,0.495961,0.167088,0.336952,quimico,quimico
Dimethicone,0.586601,0.250918,0.162482,quimico,quimico
Xylitylglucoside,0.277266,0.179188,0.543546,natural,natural
Panax Ginseng Root Extract,0.304054,0.070053,0.625893,natural,natural
Trehalose,0.174064,0.162117,0.663818,natural,natural
Serine,0.604584,0.105179,0.290238,quimico,quimico


In [ ]:
conds2 = [
    (df_topic['quimico'] > df_topic['natural'])& (df_topic['quimico']-(df_topic['natural'])>0.1),
    (df_topic['natural'] > df_topic['quimico'])& (df_topic['natural']-(df_topic['quimico'])>0.1),
    (df_topic['quimico'] > df_topic['natural']),
    (df_topic['natural'] > df_topic['quimico'])
    ]

leters2 = ['quimico','natural','no se puede determinar','no se puede determinar']

df_topic['resultado_final'] = np.select(conds2,leters)
df_topic

In [81]:
0.16/0.45

0.35555555555555557

In [7]:
from dcodebeauty.utils import predict_product

In [ ]:
predict_product('moisture bond refreshing hydra-gel oil free')